In [1]:
pip install tensorflow keras numpy pandas matplotlib nltk tqdm opencv-python flask


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/39.5 MB 2.6 MB/s eta 0:00:15
   ---------------------------------------- 0.2/39.5 MB 1.8 MB/s eta 0:00:22
   ---------------------------------------- 0.3/39.5 MB 2.2 MB/s eta 0:00:18
    --------------------------------------- 0.5/39.5 MB 2.8 MB/s eta 0:00:14
    --------------------------------------- 0.8/39.5 MB 3.2 MB/s eta 0:00:12
   - -------------------------------------- 1.1/39.5 MB 3.7 MB/s eta 0:00:11
   - -------------------------------------- 1.4/39.5 MB 4.0 MB/s eta 0:00:10
   - -------------------------------------- 1.6/39.5 MB 4.0 MB/s eta 0:00:10
   - -------------------------------------- 1.9/39.5 MB 4.1 MB/s eta 0:00:10
   -- ------------------------------------- 2.2/39.5 MB 4.1 MB/s eta 0:00:09
   -- ------------------------------------- 2.4/39.5 MB 4.2 MB/s eta 0:00:09
   -- ------------------------------------- 2.7/39.5 MB 4.1 MB/s eta 0:00:09
   ---

In [6]:
import os

annotation_file = "annotations_trainval2014/annotations/captions_train2014.json"

# Check if the file exists
if os.path.exists(annotation_file):
    print("File found!")
else:
    print("File NOT found. Check the path.")


File found!


In [5]:
annotation_file = "annotations_trainval2014/annotations/captions_train2014.json"


with open(annotation_file, 'r') as f:
    annotations = json.load(f)

print("File loaded successfully!")


File loaded successfully!


In [7]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: c:\Users\om\Desktop\Internship


In [8]:
import json
import pandas as pd

annotation_file = "annotations_trainval2014/annotations/captions_train2014.json"

with open(annotation_file, 'r') as f:
    annotations = json.load(f)

print("File loaded successfully!")


File loaded successfully!


In [9]:
import pandas as pd

# Extract image filenames and captions
image_captions = {}
for annot in annotations['annotations']:
    img_id = annot['image_id']
    img_filename = f"COCO_train2014_{img_id:012d}.jpg"  # Convert to filename format
    caption = annot['caption'].lower()  # Convert to lowercase
    
    if img_filename not in image_captions:
        image_captions[img_filename] = []
    image_captions[img_filename].append(caption)

# Convert to Pandas DataFrame
df = pd.DataFrame(list(image_captions.items()), columns=['image', 'captions'])

# Display first few rows
print(df.head())

# Save DataFrame for future use
df.to_csv("captions.csv", index=False)
print("Captions saved to captions.csv")


                             image  \
0  COCO_train2014_000000318556.jpg   
1  COCO_train2014_000000116100.jpg   
2  COCO_train2014_000000379340.jpg   
3  COCO_train2014_000000134754.jpg   
4  COCO_train2014_000000538480.jpg   

                                            captions  
0  [a very clean and well decorated empty bathroo...  
1  [a panoramic view of a kitchen and all of its ...  
2  [a graffiti-ed stop sign across the street fro...  
3  [the two people are walking down the beach., t...  
4  [a sink and a toilet inside a small bathroom.,...  
Captions saved to captions.csv


In [10]:
pip install tensorflow keras numpy tqdm opencv-python


Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tqdm import tqdm
import pickle

# Load InceptionV3 Model (pre-trained on ImageNet)
base_model = InceptionV3(weights='imagenet')
model_new = Model(base_model.input, base_model.layers[-2].output)  # Extract features before the final layer

# Function to preprocess images
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (299, 299))  # Required by InceptionV3
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

# Extract features for all images
features = {}
image_folder = "train2014/"  # Ensure this path is correct

for img_name in tqdm(df['image'][:5000]):  # Limit to 5000 images for faster training
    img_path = os.path.join(image_folder, img_name)
    if os.path.exists(img_path):  # Check if file exists
        img = preprocess_image(img_path)
        features[img_name] = model_new.predict(img)[0]

# Save extracted features
with open("image_features.pkl", "wb") as f:
    pickle.dump(features, f)

print("Feature extraction completed!")


96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 20s 0us/step


100%|██████████| 5000/5000 [00:00<00:00, 13085.41it/s]

Feature extraction completed!


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Flatten all captions into a single list
all_captions = [cap for sublist in df['captions'].values for cap in sublist]

# Tokenize captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("Tokenization completed!")


Tokenization completed!


In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Parameters
max_length = max(len(cap.split()) for cap in all_captions)  # Find max caption length
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size

# Convert captions into sequences
def create_sequences(tokenizer, captions, max_length):
    sequences = []
    for caption in captions:
        seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            sequences.append((in_seq, out_seq))
    return sequences

X_text, Y = [], []
for img_name, captions in df[['image', 'captions']].values:
    img_feature = features.get(img_name)
    if img_feature is not None:
        for in_seq, out_seq in create_sequences(tokenizer, captions, max_length):
            X_text.append(in_seq)
            Y.append(out_seq)

X_text = np.array(X_text)
Y = np.array(Y)

print("Data preparation completed!")


Data preparation completed!


In [17]:
embedding_dim = 256  # Keep embedding size consistent
units = 256  # Reduce LSTM units to match image embedding

# Image Feature Extractor (Encoder)
image_input = Input(shape=(2048,))
image_embedding = Dense(embedding_dim, activation='relu')(image_input)
image_embedding = RepeatVector(max_length)(image_embedding)  # Matches sequence length

# Captioning Model (Decoder)
text_input = Input(shape=(max_length,))
text_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(text_input)
text_embedding = LSTM(units, return_sequences=True)(text_embedding)  # Now 256 units

# Merge both inputs
decoder = Add()([image_embedding, text_embedding])  # ✅ Shapes now match
decoder = LSTM(units, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile Model
model = Model(inputs=[image_input, text_input], outputs=decoder)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 49)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │    524,544 │ input_layer_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 49, 256)   │  6,062,592 │ input_layer_7[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, 49)        │          0 │ input_layer_7[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_2     │ (None, 49, 256)   │          0 │ dense_3[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 49, 256)   │    525,312 │ embedding_2[0][0… │
│                     │                   │            │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 49, 256)   │          0 │ repeat_vector_2[… │
│                     │                   │            │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 256)       │    525,312 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 23682)     │  6,086,274 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,724,034 (52.35 MB)

 Trainable params: 13,724,034 (52.35 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Add, Input, RepeatVector
from tensorflow.keras.models import Model

# Define hyperparameters
embedding_dim = 256  # Dimension of word embeddings
units = 256  # LSTM units (kept at 256 to match image embedding)

# 🔹 Image Feature Extractor (Encoder)
image_input = Input(shape=(2048,))  # InceptionV3 gives 2048-dimensional features
image_embedding = Dense(units, activation='relu')(image_input)  # Now 256-dimensional
image_embedding = RepeatVector(max_length)(image_embedding)  # Repeat across time steps

# 🔹 Captioning Model (Decoder)
text_input = Input(shape=(max_length,))  # Input shape is the sequence length
text_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(text_input)
text_embedding = LSTM(units, return_sequences=True)(text_embedding)  # Now 256-dimensional

# 🔹 Merge both inputs
decoder = Add()([image_embedding, text_embedding])  # ✅ Now both have shape (batch_size, max_length, 256)
decoder = LSTM(units, return_sequences=False)(decoder)  # Process combined features
decoder = Dense(vocab_size, activation='softmax')(decoder)  # Predict next word

# 🔹 Compile Model
model = Model(inputs=[image_input, text_input], outputs=decoder)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# 🔹 Summary of the model
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 49)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │    524,544 │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 49, 256)   │  6,062,592 │ input_layer_9[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, 49)        │          0 │ input_layer_9[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_3     │ (None, 49, 256)   │          0 │ dense_5[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 49, 256)   │    525,312 │ embedding_3[0][0… │
│                     │                   │            │ not_equal_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 49, 256)   │          0 │ repeat_vector_3[… │
│                     │                   │            │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 256)       │    525,312 │ add_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 23682)     │  6,086,274 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,724,034 (52.35 MB)

 Trainable params: 13,724,034 (52.35 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
import os

image_folder = "train2014/"
print("Checking files in:", os.path.abspath(image_folder))

if os.path.exists(image_folder):
    total_images = len(os.listdir(image_folder))
    print(f"✅ Total Images Found: {total_images}")  # Should print 81,050
else:
    print("❌ Error: Image folder not found!")


Checking files in: c:\Users\om\Desktop\Internship\train2014
✅ Total Images Found: 81050


In [41]:
with open("image_features.pkl", "rb") as f:
    features = pickle.load(f)

print("Number of extracted image features:", len(features))  # Should be > 0


Number of extracted image features: 0


In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from PIL import Image
import os

# Load Pre-trained InceptionV3 Model
base_model = InceptionV3(weights='imagenet')
model_new = Model(base_model.input, base_model.layers[-2].output)

# Function to preprocess an image using PIL
def preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')  # Convert to RGB format
        img = img.resize((299, 299))  # Resize for InceptionV3
        img = np.array(img) / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        return preprocess_input(img)  # Apply InceptionV3 preprocessing
    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")
        return None

# Test on one image
image_folder = "train2014/"
sample_images = os.listdir(image_folder)[:1]  # Get one image

if sample_images:
    img_path = os.path.join(image_folder, sample_images[0])
    img = preprocess_image(img_path)
    
    if img is not None:
        feature = model_new.predict(img)
        print("✅ Feature extraction successful! Feature shape:", feature.shape)
    else:
        print("❌ Error: Image preprocessing failed!")
else:
    print("❌ Error: No images found in folder!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
✅ Feature extraction successful! Feature shape: (1, 2048)


In [1]:
import os
import numpy as np
import pickle
from tqdm import tqdm
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from PIL import Image

# Load Pre-trained InceptionV3 Model
base_model = InceptionV3(weights='imagenet')
model_new = Model(base_model.input, base_model.layers[-2].output)

# Function to preprocess an image using PIL
def preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')  # Convert to RGB format
        img = img.resize((299, 299))  # Resize for InceptionV3
        img = np.array(img) / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        return preprocess_input(img)  # Apply InceptionV3 preprocessing
    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")
        return None

# Dictionary to store extracted features
features = {}
image_folder = "train2014/"
image_files = os.listdir(image_folder)[:5000]  # Process first 5000 images (adjust if needed)

for img_name in tqdm(image_files, desc="Extracting Features"):
    img_path = os.path.join(image_folder, img_name)
    
    if os.path.exists(img_path):
        img = preprocess_image(img_path)
        if img is not None:
            feature = model_new.predict(img)[0]  # Extract feature
            features[img_name] = feature.tolist()  # Convert to list to avoid serialization issues
            
# Check extracted features count
print("✅ Total extracted features:", len(features))

# Save extracted features
with open("image_features.pkl", "wb") as f:
    pickle.dump(features, f)

print("✅ Feature extraction completed and saved!")


Extracting Features:   0%|          | 0/5000 [00:00<?, ?it/s]

❌ Error processing train2014/.dist: [Errno 13] Permission denied: 'C:\\Users\\om\\Desktop\\Internship\\train2014\\.dist'
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Extracting Features:   0%|          | 2/5000 [00:04<3:05:17,  2.22s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step


Extracting Features:   0%|          | 3/5000 [00:04<2:01:58,  1.46s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step


Extracting Features:  46%|████▌     | 2286/5000 [17:47<27:49,  1.63it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step


Extracting Features:  46%|████▌     | 2287/5000 [17:48<21:08,  2.14it/s]


KeyboardInterrupt: 

In [2]:
import os

image_folder = "train2014/"
image_files = os.listdir(image_folder)

print(f"✅ Total images available: {len(image_files)}")


✅ Total images available: 81051


In [3]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from PIL import Image

# Load Pre-trained InceptionV3 Model
base_model = InceptionV3(weights='imagenet')
model_new = Model(base_model.input, base_model.layers[-2].output)

# Function to preprocess an image using PIL
def preprocess_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')  # Convert to RGB format
        img = img.resize((299, 299))  # Resize for InceptionV3
        img = np.array(img) / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        return preprocess_input(img)  # Apply InceptionV3 preprocessing
    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")
        return None

# Load existing features if available (to resume)
features_file = "image_features.pkl"
if os.path.exists(features_file):
    with open(features_file, "rb") as f:
        features = pickle.load(f)
    print(f"✅ Resuming feature extraction. {len(features)} images already processed.")
else:
    features = {}

# Extract features for all images
image_folder = "train2014/"
image_files = os.listdir(image_folder)[:5000]  # Limit processing to first 5000 images

for img_name in tqdm(image_files, desc="Extracting Features"):
    if img_name in features:  # Skip already processed images
        continue

    img_path = os.path.join(image_folder, img_name)
    
    if os.path.exists(img_path):
        img = preprocess_image(img_path)
        if img is not None:
            feature = model_new.predict(img)[0]  # Extract feature
            features[img_name] = feature.tolist()  # Convert to list for serialization
            
            # Save progress every 50 images
            if len(features) % 50 == 0:
                with open(features_file, "wb") as f:
                    pickle.dump(features, f)
                print(f"✅ Saved progress at {len(features)} images.")

# Final save after completion
with open(features_file, "wb") as f:
    pickle.dump(features, f)

print(f"✅ Feature extraction completed! Total images processed: {len(features)}")


✅ Resuming feature extraction. 0 images already processed.


Extracting Features:   0%|          | 0/5000 [00:00<?, ?it/s]

❌ Error processing train2014/.dist: [Errno 13] Permission denied: 'C:\\Users\\om\\Desktop\\Internship\\train2014\\.dist'
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


Extracting Features:   0%|          | 2/5000 [00:04<3:00:31,  2.17s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step


Extracting Features:   7%|▋         | 369/5000 [03:21<32:24,  2.38it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step


Extracting Features:   7%|▋         | 370/5000 [03:21<42:05,  1.83it/s]


KeyboardInterrupt: 

In [5]:
# Extract features for all images
image_folder = "train2014/"
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]  # ✅ Only process images
image_files = image_files[:5000]  # Process first 5000 images

for img_name in tqdm(image_files, desc="Extracting Features"):
    if img_name in features:  # Skip already processed images
        continue

    img_path = os.path.join(image_folder, img_name)
    
    if os.path.exists(img_path):
        img = preprocess_image(img_path)
        if img is not None:
            feature = model_new.predict(img)[0]  # Extract feature
            features[img_name] = feature.tolist()  # Convert to list for serialization
            
            # Save progress every 50 images
            if len(features) % 50 == 0:
                with open(features_file, "wb") as f:
                    pickle.dump(features, f)
                print(f"✅ Saved progress at {len(features)} images.")

# Final save after completion
with open(features_file, "wb") as f:
    pickle.dump(features, f)

print(f"✅ Feature extraction completed! Total images processed: {len(features)}")


Extracting Features:   0%|          | 0/5000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step


Extracting Features:   7%|▋         | 370/5000 [00:00<00:04, 1011.93it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

Extracting Features:   7%|▋         | 370/5000 [00:13<00:04, 1011.93it/s]

...

Extracting Features: 100%|█████████▉| 4999/5000 [34:18<00:00,  2.01it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step


Extracting Features: 100%|██████████| 5000/5000 [34:19<00:00,  2.43it/s]

✅ Saved progress at 5000 images.


✅ Feature extraction completed! Total images processed: 5000


In [ ]:
import os
import pickle
import gc
from tqdm import tqdm
import numpy as np

image_folder = "train2014/"
features_file = "image_features.pkl"

# Load existing features if available
if os.path.exists(features_file):
    with open(features_file, "rb") as f:
        features = pickle.load(f)
else:
    features = {}

image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Only process images not yet extracted
image_files = [img for img in image_files if img not in features]

print(f"✅ Resuming extraction. {len(image_files)} images left to process.")

for img_name in tqdm(image_files, desc="Extracting Features"):
    img_path = os.path.join(image_folder, img_name)

    try:
        if os.path.exists(img_path):
            img = preprocess_image(img_path)  # Ensure preprocess_image() handles errors
            if img is not None:
                feature = model_new.predict(img)[0]  # Extract feature
                features[img_name] = feature.tolist()  # Store feature

                # Save progress every 50 images
                if len(features) % 50 == 0:
                    with open(features_file, "wb") as f:
                        pickle.dump(features, f)
                    print(f"✅ Saved progress at {len(features)} images.")
                    
                # **Free memory**
                del img, feature
                gc.collect()
    
    except Exception as e:
        print(f"❌ Error processing {img_name}: {str(e)}")  # Skip and continue

# Final save
with open(features_file, "wb") as f:
    pickle.dump(features, f)

print(f"✅ Feature extraction completed! Total images processed: {len(features)}")


✅ Resuming extraction. 76050 images left to process.


Extracting Features:   0%|          | 0/76050 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step


Extracting Features:   0%|          | 1/76050 [00:01<22:53:30,  1.08s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 780ms/step


Extracting Features:   1%|▏         | 983/76050 [14:46<28:49:54,  1.38s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


Extracting Features:   1%|▏         | 984/76050 [14:50<41:49:28,  2.01s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
